In [1]:
import torch
from torch.autograd import Function, Variable
import torch.nn.functional as F
from torch import nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from torch.nn.utils import parameters_to_vector
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import numpy.random as npr

import matplotlib
from matplotlib import pyplot as plt

from mpc import mpc
from mpc.mpc import GradMethods, QuadCost, LinDx
from mpc import casadi_control
#from mpc.dynamics import NNDynamics
#import mpc.util as eutil
from mpc.env_dx import frenet_kin_bicycle
from mpc.track.src import simple_track_generator, track_functions

import time
import os
import shutil
import pickle as pkl
import collections

import argparse

In [2]:
# Let's try to create a track 

track_density = 300
track_width = 0.5
v_max = 2
gen = simple_track_generator.trackGenerator(track_density,track_width)
track_name = 'LONG_TRACK'

In [3]:
t = 0.3
init = [0,0,0]

track_function = {
    'DEMO_TRACK'    : track_functions.demo_track,
    'HARD_TRACK'    : track_functions.hard_track,
    'LONG_TRACK'    : track_functions.long_track,
    'LUCERNE_TRACK' : track_functions.lucerne_track,
    'BERN_TRACK'    : track_functions.bern_track,
    'INFINITY_TRACK': track_functions.infinity_track,
    'SNAIL_TRACK'   : track_functions.snail_track
}.get(track_name, track_functions.demo_track)
    
track_function(gen, t, init)
    
gen.populatePointsAndArcLength()
gen.centerTrack()

track_coord = torch.from_numpy(np.vstack([gen.xCoords, gen.yCoords, gen.arcLength, gen.tangentAngle, gen.curvature]))

In [4]:
device = 'cpu'

def sample_xinit(n_batch):
    def uniform(shape, low, high):
        r = high-low
        return torch.rand(shape)*r+low
    sigma = uniform(n_batch, 0.01, 10.)
    d = uniform(n_batch, -0.05, 0.05)
    phi = uniform(n_batch, -0.15*np.pi, 0.15*np.pi)
    v = uniform(n_batch, 0., 0.5)
    sigma_0 = sigma
    sigma_diff = sigma-sigma_0
    d_lb = softplus_op(-d-0.5*track_width)
    d_ub = softplus_op(d-0.5*track_width)
    #_lb = softplus_op(-v + 0)
    v_ub = softplus_op(v-v_max)
    xinit = torch.stack((sigma, d, phi, v, sigma_0, sigma_diff, d_lb, d_ub, v_ub), dim=1)
    return xinit


true_dx = frenet_kin_bicycle.FrenetKinBicycleDx(track_coord)
mpc_T = 20
n_batch = 8
H_curve = 60
use_NN_curv = True

# Added here the bounds of U
u_lower = torch.tensor([-2., -0.5]).unsqueeze(0).unsqueeze(0).repeat(mpc_T, n_batch, 1)
u_upper = torch.tensor([2., 0.5]).unsqueeze(0).unsqueeze(0).repeat(mpc_T, n_batch, 1)
u_init= torch.tensor([0.1, 0.0]).unsqueeze(0).unsqueeze(0).repeat(mpc_T, n_batch, 1)


n_state = true_dx.n_state
n_ctrl = true_dx.n_ctrl

eps = 10
lqr_iter = 10
grad_method = GradMethods.AUTO_DIFF

softplus_op = torch.nn.Softplus(10)

8


In [5]:
def get_nearest_index(point_f, ref_path):
    return ((point_f[0] - ref_path[2,:])**2).argmin()
    
def compute_x_coord(point_f, ref_path, nearest_index):
    return ref_path[0,nearest_index] - point_f[1]*torch.sin(ref_path[3,nearest_index])

def compute_y_coord(point_f, ref_path, nearest_index):
    return ref_path[1,nearest_index] + point_f[1]*torch.cos(ref_path[3,nearest_index])

def frenet_to_cartesian(point_f, ref_path):     
    nearest_index = get_nearest_index(point_f, ref_path)
    x = compute_x_coord(point_f, ref_path, nearest_index)
    y = compute_y_coord(point_f, ref_path, nearest_index)
    
    return torch.tensor([x, y])

In [6]:
# casadi mpc with exact penalty
test_q = np.array([ 0.,  6.,  1.,  0., 0., 0. , 0., 0., 1., 2.])
test_p = np.array([ 0.,  0.,  0.,  0., 0., -2., 100., 100.,  -1,  0.])

control = casadi_control.CasadiControl(track_coord)

x0 = (sample_xinit(1)).numpy()
print(x0[0])
dc = 2 #number constraints
df = 2 #number of states we artificially added like sigma_0, sigma_diff
dx = n_state #number states taken from the dynamics (including penalties and sigma_o/sigma_diff)
du = n_ctrl #number control inputs
horizon = mpc_T
sol = control.mpc_casadi_with_constraints(test_q,test_p,x0,horizon,df,dc,dx,du)
N = horizon
u = sol[-du*N:]
x = sol[:-du*N]
u_r = u.reshape(N,du)
x_r = x.reshape(N+1,dx-dc-df)
print(u_r)
print(x_r)

[ 1.6466508e+00  3.4642454e-02 -2.5744149e-01  3.6215094e-01
  1.6466508e+00  0.0000000e+00  5.6429016e-03  1.0981217e-02
  7.7074720e-09]
curv start
curv end
curv start
curv end
curv start
curv end

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      404
Number of nonzeros in inequality constraint Jacobian.:       82
Number of nonzeros in Lagrangian Hessian.............:      300

Total number of variables............................:      124
                     variables with only lower bounds:        0
                varia